In [13]:
import os
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent)

#### Load Dataset

In [14]:
from EmoBox.EmoBox import EmoDataset, EmoEval

In [15]:
dataset = "cremad"
fold = 1  # different datasets have different number of folds, which can be find in data/
user_data_dir = "./" # path to EmoBox - FIXED: Changed from "Emobox" to "EmoBox"
meta_data_dir = "EmoBox/data/" # path to data folder - FIXED: Changed from "Emobox" to "EmoBox"


train = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="train")
test = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="test")

since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 5586 samples, only 5586 exists in data dir EmoBox/data/
load in 1856 samples, only 1856 exists in data dir EmoBox/data/
Num. training samples 5586
Num. valid samples 0
Num. test samples 1856
Using label_map {'ANG': 'Angry', 'DIS': 'Disgust', 'FEA': 'Fear', 'HAP': 'Happy', 'NEU': 'Neutral', 'SAD': 'Sad'}
since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 5586 samples, only 5586 exists in data dir EmoBox/data/
load in 1856 samples, only 1856 exists in data dir EmoBox/data/
Num. training samples 5586
Num. valid samples 0
Num. test samples 1856
Using label_map {'ANG': 'Angry', 'DIS': 'Disgust', 'FEA': 'Fear', 'HAP': 'Happy', 'NEU': 'Neutral', 'SAD': 'Sad'}


In [16]:
sample = test[0]
sample

{'key': 'cremad-1052-WSI-ANG-XX',
 'audio': array([-0.0055542 , -0.00469971, -0.00601196, ...,  0.        ,
         0.        ,  0.        ], shape=(36837,), dtype=float32),
 'label': 'Angry',
 'gender': 'Female',
 'age': 33,
 'race': 'Caucasian',
 'ethnicity': 'Not Hispanic',
 'language': 'English'}

In [17]:
test.label_map.values()

dict_values(['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad'])

In [18]:
from collections import Counter
labels =  [data['label'] for data in test]
Counter(labels)

Counter({'Angry': 317,
         'Sad': 317,
         'Fear': 317,
         'Happy': 317,
         'Disgust': 317,
         'Neutral': 271})

#### Load Model

In [7]:
import torch
from mllm_emotion_classifier.models import ModelFactory

device = "cuda" if torch.cuda.is_available() else "cpu"
model = ModelFactory.create(
    name="qwen2-audio-instruct",
    class_labels=set(train.label_map.values()),
    do_sample=False,
    prompt_name="user_labels",
    device=device
)

/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|███████████████████████████████████████████████| 876/876 [00:02<00:00, 299.82it/s, Materializing param=multi_modal_projector.linear.weight]


In [8]:
data_loader = torch.utils.data.DataLoader(
    dataset=test,
    batch_size=4,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
    collate_fn=model.collate_fn
)

In [9]:
# from tqdm import tqdm
# predictions, labels = [], []
# i = 0
# for inputs, lbl in tqdm(data_loader, total=len(data_loader)):
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     preds = model.predict(inputs)
#     predictions.extend(preds)
#     labels.extend(lbl)
#     i += 1
#     if i == 100: break

#### Evaluation

In [10]:
from mllm_emotion_classifier.evaluate import Evaluator

evaluator = Evaluator()
evaluator.evaluate(model, data_loader)

/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/src/mllm_emotion_classifier/evaluate/evaluate.py:96: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  valid_indices = [i for i, p in enumerate(self.y_pred) if p is not "Unknown"]



Evaluating qwen2-audio-instruct on emovdb


Inference:   4%|████▍                                                                                                           | 17/430 [00:09<03:15,  2.11it/s]Could not confidently parse response: "Happy"
Could not confidently parse response: "Happy"
Inference:  11%|███████████▉                                                                                                    | 46/430 [00:22<02:58,  2.15it/s]Could not confidently parse response: "Happy"
Could not confidently parse response: "Happy"
Inference: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 430/430 [03:16<00:00,  2.19it/s]
/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/.venv/lib/python3.10/site-packages/skl

{'timestamp': '2025-12-19 17:25:23',
 'dataset': 'emovdb',
 'model_name': 'qwen2-audio-instruct',
 'fold': None,
 'num_samples': 1719,
 'valid_rate': 1.0,
 'class_labels': ['Neutral', 'Disgust', 'Angry', 'Amused', 'Sleepy'],
 'metrics': {'global': {'f1_macro': 0.6818,
   'f1_weighted': 0.8191,
   'accuracy_unweighted': 0.8261,
   'accuracy_weighted': 0.8086,
   'precision_macro': 0.7014,
   'precision_weighted': 0.8589,
   'recall_macro': 0.6885,
   'recall_weighted': 0.8086},
  'classwise': {'accuracy': {'Amused': 0.9331,
    'Angry': 0.9796,
    'Disgust': 0.8941,
    'Neutral': 0.9191,
    'Sleepy': 0.9092},
   'false_positive_rate': {'Amused': 0.0266,
    'Angry': 0.0107,
    'Disgust': 0.1188,
    'Neutral': 0.0459,
    'Sleepy': 0.0085},
   'false_negative_rate': {'Amused': 0.2378,
    'Angry': 0.0633,
    'Disgust': 0.0315,
    'Neutral': 0.1995,
    'Sleepy': 0.3372},
   'true_positive_rate': {'Amused': 0.7622,
    'Angry': 0.9367,
    'Disgust': 0.9685,
    'Neutral': 0.8005,


In [11]:
evaluator.results['metrics']['global']

{'f1_macro': 0.6818,
 'f1_weighted': 0.8191,
 'accuracy_unweighted': 0.8261,
 'accuracy_weighted': 0.8086,
 'precision_macro': 0.7014,
 'precision_weighted': 0.8589,
 'recall_macro': 0.6885,
 'recall_weighted': 0.8086}